**Installation**

In [43]:
pip install pyspark

In [44]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [45]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()

In [46]:
sc=spark.sparkContext

Read Data

In [47]:
text = spark.read.text("/kaggle/input/textsc/*")
text=text.rdd

Clean Data

In [48]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;“”<=>?@[\\]^_`{|}~-1234567890'
  lowercased_str = x.value.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [49]:
text_new = text.map(lower_clean_str)

In [50]:
text_count = text_new.flatMap(lambda x: x.split(' '))\
                .map(lambda x: (x, 1))

In [51]:
text_count = text_count.reduceByKey(lambda x, y: x+y)

In [52]:
text_count.count()

In [53]:
df1=text_count.toDF().sort('_2',ascending=False).toPandas()

In [54]:
df1=df1.rename(columns={"_1": "word", "_2": "cnt"})

Stopwords

In [55]:
import nltk
from nltk.corpus import stopwords

In [56]:
nltk.download('stopwords')

In [57]:
stopwords=stopwords.words('english')

In [58]:
df1 = df1[~df1['word'].isin(stopwords)]
n=10
result=df1.iloc[2:].head(n)

In [59]:
df1.head(n)

**Wordcloud Vizualization**

In [60]:
result.to_csv(f"top{n}_words.csv")

In [61]:
result['word']=result['word'].astype(str)

In [62]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

text = ' '.join(result.word.values)
wordcloud = WordCloud(width = 2000, height = 1000, random_state=1, collocations=False, background_color="white").generate(text)
plt.figure(figsize=(30, 20))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()